![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)



<p><font size="5" color='grey'> <b>
Multimodales RAG - Retrieval Augmented Generation
</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('markitdown[all]', 'markitdown'),
    'langchain_chroma',
])

In [ ]:
#@title 📂 Dokumente & Bilder kopieren { display-mode: "form" }
!rm -rf files
!mkdir files
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_1.txt -o files/biografien_1.txt
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_2.md -o files/biografien_2.md
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_3.pdf -o files/biografien_3.pdf
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_4.docx -o files/biografien_4.docx
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/a_retro-futuristic_robot_dall_e.jpg -o files/a_retro-futuristic_robot_dall_e.jpg
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/hedra_cyborg.png -o files/hedra_cyborg.png
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/apfel.jpg -o files/apfel.jpg

# 1 | Funktionale Architektur
---



<p><font color='black' size="5">
Aufbau des funktionalen RAG-Systems
</font></p>

Das funktionale System besteht aus drei Hauptkomponenten:

**1. Konfiguration & Setup:**
- `RAGConfig`: Zentrale Konfiguration als Dataclass
- `RAGComponents`: Container für alle System-Komponenten
- `init_rag_system()`: Einmalige Initialisierung

**2. Dokumenten-Verarbeitung:**
- `add_text_document()`: Einzelnes Text-Dokument hinzufügen
- `add_image()`: Einzelnes Bild hinzufügen  
- `process_directory()`: Gesamtes Verzeichnis verarbeiten

**3. Suche & Abfrage:**
- `search_texts()`: Reine Text-Suche
- `search_images()`: Reine Bild-Suche
- `multimodal_search()`: Kombinierte Suche

**Datenfluss:**
```
Dokumente → Verarbeitung → Embeddings → Vektordatenbank → Suche → LLM → Antwort
```



# 2 | Implementierung
---




<p><font color='black' size="5">
Importe und Grundkonfiguration
</font></p>

In [ ]:
from pathlib import Path
from typing import List, Dict, Any, Tuple
import uuid
from dataclasses import dataclass

from markitdown import MarkItDown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer
from PIL import Image
import chromadb


<p><font color='black' size="5">
Konfiguration und Datenstrukturen
</font></p>

In [ ]:
@dataclass
class RAGConfig:
    """Zentrale Konfiguration für das RAG-System"""
    chunk_size: int = 200
    chunk_overlap: int = 20
    text_threshold: float = 1.2
    image_threshold: float = 0.8
    clip_model: str = 'clip-ViT-B-32'
    text_model: str = 'text-embedding-3-small'
    llm_model: str = 'gpt-4o-mini'
    db_path: str = './multimodal_rag_db'

@dataclass
class RAGComponents:
    """Container für alle RAG-System-Komponenten"""
    text_embeddings: OpenAIEmbeddings
    clip_model: SentenceTransformer
    llm: ChatOpenAI
    text_splitter: RecursiveCharacterTextSplitter
    markitdown: MarkItDown
    chroma_client: chromadb.PersistentClient
    text_collection: Chroma
    image_collection: Any


<p><font color='black' size="5">
System-Initialisierung
</font></p>

In [ ]:
def init_rag_system(config: RAGConfig = RAGConfig()) -> RAGComponents:
    """Initialisiert alle RAG-Komponenten"""
    print(f"🚀 Initialisiere RAG-System in {config.db_path}")

    # KI-Modelle laden
    text_embeddings = OpenAIEmbeddings(model=config.text_model)
    print("✅ OpenAI Text-Embeddings initialisiert")

    print("🖼️ Lade CLIP-Modell...")
    clip_model = SentenceTransformer(config.clip_model)
    print("✅ CLIP-Modell geladen")

    llm = ChatOpenAI(model=config.llm_model, temperature=0)

    # Text-Verarbeitung
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config.chunk_size,
        chunk_overlap=config.chunk_overlap
    )
    markitdown = MarkItDown()

    # Datenbank einrichten
    Path(config.db_path).mkdir(exist_ok=True)
    chroma_client = chromadb.PersistentClient(path=config.db_path)

    # Text-Collection
    text_collection = Chroma(
        collection_name="texts",
        embedding_function=text_embeddings,
        persist_directory=config.db_path
    )

    # Bild-Collection
    collections = [c.name for c in chroma_client.list_collections()]
    if "images" in collections:
        image_collection = chroma_client.get_collection("images")
    else:
        image_collection = chroma_client.create_collection(
            name="images", metadata={"hnsw:space": "cosine"}
        )

    print("✅ Collections initialisiert")

    return RAGComponents(
        text_embeddings, clip_model, llm, text_splitter,
        markitdown, chroma_client, text_collection, image_collection
    )


<p><font color='black' size="5">
Text-Dokument hinzufügen
</font></p>

In [ ]:
def add_text_document(components: RAGComponents, file_path: str) -> bool:
    """Fügt ein Text-Dokument zur Datenbank hinzu"""
    path = Path(file_path).absolute()

    # Duplikatsprüfung
    if components.text_collection.get(where={"source": str(path)})['ids']:
        print(f"⚠️ {path.name} bereits vorhanden")
        return False

    try:
        # Dokument mit MarkItDown konvertieren
        result = components.markitdown.convert(str(path))
        if not result or not result.text_content.strip():
            print(f"⚠️ {path.name} enthält keinen Text")
            return False

        # Text in Chunks aufteilen
        chunks = components.text_splitter.split_text(result.text_content)
        documents = [
            Document(
                page_content=chunk.strip(),
                metadata={"source": str(path), "filename": path.name, "chunk_id": i}
            ) for i, chunk in enumerate(chunks) if chunk.strip()
        ]

        # Zur Datenbank hinzufügen
        if documents:
            components.text_collection.add_documents(documents)
            print(f"✅ {len(documents)} Chunks von '{path.name}' hinzugefügt")
            return True

    except Exception as e:
        print(f"❌ Fehler bei {path.name}: {e}")

    return False


<p><font color='black' size="5">
Bild hinzufügen
</font></p>

In [ ]:
def add_image(components: RAGComponents, image_path: str, description: str = "") -> bool:
    """Fügt ein Bild zur Datenbank hinzu"""
    path = Path(image_path).absolute()

    if not path.exists():
        print(f"❌ Bild nicht gefunden: {path}")
        return False

    # Duplikatsprüfung
    if components.image_collection.get(where={"source": str(path)})['ids']:
        print(f"⚠️ Bild bereits vorhanden: {path.name}")
        return False

    try:
        # Bild laden und Embedding erstellen
        image = Image.open(path).convert('RGB')
        embedding = components.clip_model.encode(image).tolist()
        print(f"🖼️ Bild-Embedding erstellt für {path.name}")

        # Metadaten vorbereiten
        doc_id = f"img_{uuid.uuid4().hex[:8]}_{path.name}"
        content = f"Bild: {path.name}"
        if description.strip():
            content += f" - {description.strip()}"

        # Zur Datenbank hinzufügen
        components.image_collection.add(
            ids=[doc_id],
            embeddings=[embedding],
            documents=[content],
            metadatas=[{
                "source": str(path),
                "filename": path.name,
                "description": description.strip()
            }]
        )

        print(f"✅ Bild '{path.name}' hinzugefügt")
        return True

    except Exception as e:
        print(f"❌ Fehler bei Bild {path.name}: {e}")
        return False


<p><font color='black' size="5">
Verzeichnis verarbeiten
</font></p>

In [ ]:
def process_directory(components: RAGComponents, directory: str, include_images: bool = True) -> Dict[str, int]:
    """Verarbeitet alle Dateien in einem Verzeichnis"""
    dir_path = Path(directory)
    if not dir_path.exists():
        print(f"❌ Verzeichnis nicht gefunden: {directory}")
        return {"texts": 0, "images": 0}

    # Unterstützte Dateitypen
    text_extensions = {'.pdf', '.docx', '.txt', '.md', '.html'}
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp'}

    # Dateien sammeln
    text_files = [f for f in dir_path.rglob("*") if f.suffix.lower() in text_extensions]
    image_files = [f for f in dir_path.rglob("*") if f.suffix.lower() in image_extensions] if include_images else []

    print(f"📊 Gefunden: {len(text_files)} Text-Dateien, {len(image_files)} Bilder")

    # Text-Dateien verarbeiten
    text_count = 0
    for file_path in text_files:
        print(f"📄 {file_path.name}")
        if add_text_document(components, str(file_path)):
            text_count += 1

    # Bild-Dateien verarbeiten
    image_count = 0
    for img_path in image_files:
        print(f"🖼️ {img_path.name}")
        # Automatische Beschreibung aus Dateiname
        description = img_path.stem.replace('_', ' ').replace('-', ' ')
        if add_image(components, str(img_path), description):
            image_count += 1

    return {"texts": text_count, "images": image_count}

<p><font color='black' size="5">
Text-Suche
</font></p>

In [ ]:
def search_texts(components: RAGComponents, query: str, k: int = 3, config: RAGConfig = RAGConfig()) -> str:
    """ Durchsucht Text-Dokumente mit einheitlicher Ähnlichkeits-Terminologie """

    if not components.text_collection.get()['ids']:
        return "❌ Keine Text-Dokumente gefunden"

    # Ähnlichkeitssuche durchführen
    docs_with_scores = components.text_collection.similarity_search_with_score(query, k=k*2)
    if not docs_with_scores:
        return "❌ Keine relevanten Dokumente gefunden"

    # Score in Ähnlichkeit umwandeln (0-1, wobei 1 = identisch)
    docs_with_similarity = []
    for doc, score in docs_with_scores:
        # Annahme: Score ist Distanz (niedrig = ähnlich)
        # Umwandlung in Ähnlichkeit mit exponentieller Dämpfung für bessere Verteilung
        similarity = max(0, min(1, 2.0 / (1 + score)))
        docs_with_similarity.append((doc, similarity))

    # Nach Ähnlichkeit sortieren (höchste zuerst)
    docs_with_similarity.sort(key=lambda x: x[1], reverse=True)

    # Top k Dokumente nehmen, die einen Mindest-Ähnlichkeitswert erreichen
    min_similarity = 0.3  # Anpassbarer Schwellenwert (entspricht etwa score < 1.2 im Original)
    relevant_docs = [(doc, sim) for doc, sim in docs_with_similarity[:k]
                     if sim >= min_similarity]

    if not relevant_docs:
        return "❌ Keine ausreichend ähnlichen Dokumente gefunden"

    # Kontext für LLM zusammenstellen
    context = "\n\n---\n\n".join([doc.page_content for doc, _ in relevant_docs])
    sources = [
        {
            "filename": doc.metadata.get("filename", "Unbekannt"),
            "similarity": round(sim, 3)
        }
        for doc, sim in relevant_docs
    ]

    # LLM-Antwort generieren
    prompt = f"""Beantworte die Frage präzise basierend auf dem Kontext.

KONTEXT:
{context}

FRAGE: {query}

ANTWORT:"""

    response = components.llm.invoke(prompt).content

    # Einheitliche Ausgabe mit Ähnlichkeits-Terminologie
    source_text = f"\n\n📚 Quellen ({len(sources)}): " + "\n".join([
        f"   • {src['filename']} (Ähnlichkeit: {src['similarity']})" for src in sources
    ])

    return f"{response}{source_text}"


<p><font color='black' size="5">
Bild-Suche
</font></p>

In [ ]:
def search_images(components: RAGComponents, query: str, k: int = 3, config: RAGConfig = RAGConfig()) -> List[Dict[str, Any]]:
    """Durchsucht Bilder mit Text-Query"""
    if components.image_collection.count() == 0:
        return []

    # Text-Query in Bild-Embedding-Raum umwandeln
    query_embedding = components.clip_model.encode(query).tolist()

    # Suche in Bild-Collection
    results = components.image_collection.query(
        query_embeddings=[query_embedding],
        n_results=min(k*2, components.image_collection.count()),
        include=['documents', 'metadatas', 'distances']
    )

    if not results['ids'][0]:
        return []

    # Ergebnisse filtern und formatieren
    return [
        {
            "filename": metadata.get("filename", "Unbekannt"),
            "path": metadata.get("source", ""),
            "description": metadata.get("description", ""),
            "similarity": round(max(0, 1 - distance), 3)
        }
        for distance, metadata in zip(results['distances'][0], results['metadatas'][0])
        if distance < config.image_threshold
    ]


<p><font color='black' size="5">
Multimodale Suche
</font></p>

In [ ]:
def multimodal_search(components: RAGComponents, query: str, k_text: int = 3, k_images: int = 3) -> str:
    """Führt multimodale Suche durch (Text + Bilder)"""
    mprint(f"## 🔍 Multimodale Suche: {query}")

    # Parallele Suche in beiden Modalitäten
    text_results = search_texts(components, query, k_text)
    image_results = search_images(components, query, k_images)

    # Ergebnisse zusammenfassen
    result = f"### 📄 TEXT-ERGEBNISSE:\n{text_results}\n\n"

    if image_results:
        result += f"### 🖼️ BILD-ERGEBNISSE ({len(image_results)} gefunden):\n"
        for i, img in enumerate(image_results, 1):
            result += f"   {i}. {img['filename']} (Ähnlichkeit: {img['similarity']})\n"
            if img['description']:
                result += f"      📝 {img['description']}\n"
    else:
        result += "🖼️ Keine relevanten Bilder gefunden.\n"

    return result


<p><font color='black' size="5">
Hilfsfunktionen
</font></p>

In [ ]:
def get_system_status(components: RAGComponents) -> Dict[str, Any]:
    """Gibt detaillierten System-Status zurück"""
    text_data = components.text_collection.get()
    text_count = len(text_data['ids'])
    image_count = components.image_collection.count()

    # print(f"📊 Status: {text_count} Text-Chunks, {image_count} Bilder")

    return {
        "text_chunks": text_count,
        "images": image_count,
        "total_documents": text_count + image_count
    }

def cleanup_database(db_path: str = './multimodal_rag_db'):
    """Löscht die Datenbank komplett"""
    import shutil
    if Path(db_path).exists():
        shutil.rmtree(db_path)
        print(f"🗑️ Datenbank gelöscht: {db_path}")

# 3 | Hands-On: Multimodales RAG
---


## 3.1 *Alte* Datenbank löschen

In [ ]:
# Aufräumen vor Neustart
cleanup_database('./multimodal_rag_db')

## 3.2 System initialisieren

In [ ]:
# Konfiguration erstellen
config = RAGConfig(db_path='./multimodal_rag_db')

# RAG-System initialisieren
rag_components = init_rag_system(config)

## 3.3 Dokumente verarbeiten

In [ ]:
# Alle Dateien im files-Verzeichnis verarbeiten
results = process_directory(rag_components, './files', include_images=True)

mprint(f"### 📊 Verarbeitungsergebnis:")
print(f"   📄 Text-Dokumente: {results['texts']}")
print(f"   🖼️ Bilder: {results['images']}")
print(f"   📦 Gesamt: {results['texts'] + results['images']}")

## 3.4 System-Status anzeigen

In [ ]:
# Aktueller Status der Datenbank
status = get_system_status(rag_components)
mprint(f"### Datenbank enthält:")
print(f"   📄 {status['text_chunks']} Text-Chunks")
print(f"   🖼️ {status['images']} Bilder")
print(f"   📦 {status['total_documents']} Dokumente insgesamt")

## 3.5 Text-Suche testen

In [ ]:
# Reine Text-Suche
query = "Wer ist Thoren Navarro?"
result = search_texts(rag_components, query)

mprint(f"### 🔍 TEXT-SUCHE")
mprint(f"**Query:** {query}")
mprint("---")
mprint(result)

## 3.6 Bild-Suche testen

In [ ]:
# Reine Bild-Suche
query = "Roboter"
images = search_images(rag_components, query)

mprint(f"### 🖼️ BILD-SUCHE")
mprint(f"**Query:** {query}")
mprint("---")

if images:
    mprint(f"**{len(images)} Bilder gefunden:**")
    for i, img in enumerate(images, 1):
        mprint(f"{i}. **{img['filename']}** (Ähnlichkeit: {img['similarity']})")
        if img['description']:
            mprint(f"   📝 *{img['description']}*")
else:
    mprint("Keine relevanten Bilder gefunden.")

## 3.7 Multimodale Suche testen

In [ ]:
# Kombinierte Text- und Bild-Suche
test_queries = [
    "Wer ist Thoren Navarro?",
    "Zeige mir Bilder von Robotern",
    "Was sind KI-Modelle?",
    "Futuristische Technologien"
]

for query in test_queries:
    print()
    result = multimodal_search(rag_components, query, k_text=2, k_images=2)
    mprint(result)

# 4 | Erweiterte Funktionen
---




<p><font color='black' size="5">
Batch-Verarbeitung
</font></p>

In [ ]:
def process_multiple_directories(components: RAGComponents, directories: List[str]) -> Dict[str, Any]:
    """Verarbeitet mehrere Verzeichnisse in einem Durchgang"""
    total_results = {"texts": 0, "images": 0}

    for directory in directories:
        print(f"\n📁 Verarbeite Verzeichnis: {directory}")
        results = process_directory(components, directory, include_images=True)
        total_results["texts"] += results["texts"]
        total_results["images"] += results["images"]

    return total_results

# Usage
directories = ['./docs', './images', './pdfs']
# total = process_multiple_directories(rag_components, directories)


<p><font color='black' size="5">
Erweiterte Suchoptionen
</font></p>

In [ ]:
def advanced_search(components: RAGComponents, query: str, filters: Dict[str, Any] = None) -> Dict[str, Any]:
    """Erweiterte Suche mit Filtern"""
    filters = filters or {}

    # Text-Suche mit optionalen Filtern
    text_results = []
    if not filters.get('images_only', False):
        text_results = search_texts(components, query, k=filters.get('k_text', 3))

    # Bild-Suche mit optionalen Filtern
    image_results = []
    if not filters.get('text_only', False):
        image_results = search_images(components, query, k=filters.get('k_images', 3))

    return {
        "query": query,
        "text_results": text_results,
        "image_results": image_results,
        "total_found": len(text_results) + len(image_results)
    }

# Usage Examples
# result = advanced_search(rag_components, "KI", {"text_only": True})
# result = advanced_search(rag_components, "Roboter", {"images_only": True})


<p><font color='black' size="5">
Performance-Monitoring
</font></p>

In [ ]:
import time
from functools import wraps

def measure_time(func):
    """Decorator für Performance-Messung"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        print(f"⏱️ {func.__name__} dauerte {duration:.2f}s")
        return result
    return wrapper

@measure_time
def timed_multimodal_search(components: RAGComponents, query: str) -> str:
    """Multimodale Suche mit Zeitmessung"""
    return multimodal_search(components, query)

# Usage
result = timed_multimodal_search(rag_components, "Test Query")

# 5 | Performance Test
---


In [ ]:
import time
from typing import Callable

def benchmark_function(func: Callable, *args, iterations: int = 10) -> Dict[str, float]:
    """Benchmarkt eine Funktion über mehrere Iterationen"""
    times = []

    for _ in range(iterations):
        start = time.time()
        func(*args)
        end = time.time()
        times.append(end - start)

    return {
        "avg_time": sum(times) / len(times),
        "min_time": min(times),
        "max_time": max(times),
        "total_time": sum(times)
    }

# Usage Example
benchmark_result = benchmark_function(search_texts, rag_components, "test query")
print(f"Durchschnittliche Suchzeit: {benchmark_result['avg_time']:.3f}s")

# 6 | Gradio Interface
---

In [ ]:
import gradio as gr

def create_gradio_interface(components: RAGComponents) -> gr.Blocks:
    """Erstellt eine benutzerfreundliche Web-Oberfläche"""

    with gr.Blocks(title="Funktionales Multimodales RAG") as interface:
        gr.HTML("<h1>🔍 Multimodales RAG System</h1>")

        with gr.Tab("Suchen"):
            with gr.Row():
                with gr.Column(scale=2):
                    query_input = gr.Textbox(
                        label="Ihre Frage",
                        placeholder="z.B. 'Wer ist Thoren Navarro?' oder 'Zeige mir Roboter-Bilder'",
                        lines=3
                    )

                    with gr.Row():
                        search_btn = gr.Button("🔍 Suchen", variant="primary")
                        clear_btn = gr.Button("🗑️ Löschen")

                with gr.Column(scale=1):
                    search_type = gr.Radio(
                        choices=["multimodal", "text", "images"],
                        value="multimodal",
                        label="Suchtyp"
                    )
                    k_text = gr.Slider(1, 10, 3, label="Anzahl Text-Ergebnisse")
                    k_images = gr.Slider(1, 10, 3, label="Anzahl Bild-Ergebnisse")

            result_output = gr.Markdown(label="Ergebnisse")

            def perform_search(query, stype, kt, ki):
                if not query.strip():
                    return "❌ Bitte geben Sie eine Frage ein."

                try:
                    if stype == "text":
                        return search_texts(components, query, int(kt))
                    elif stype == "images":
                        images = search_images(components, query, int(ki))
                        if images:
                            result = f"### 🖼️ {len(images)} Bilder gefunden:\n"
                            for i, img in enumerate(images, 1):
                                result += f"{i}. **{img['filename']}** (Ähnlichkeit: {img['similarity']})\n"
                                if img['description']:
                                    result += f"   📝 *{img['description']}*\n"
                            return result
                        else:
                            return "❌ Keine relevanten Bilder gefunden."
                    else:  # multimodal
                        return multimodal_search(components, query, int(kt), int(ki))

                except Exception as e:
                    return f"❌ Fehler bei der Suche: {str(e)}"

            search_btn.click(
                perform_search,
                inputs=[query_input, search_type, k_text, k_images],
                outputs=result_output
            )

            clear_btn.click(
                lambda: ("", ""),
                outputs=[query_input, result_output]
            )

        with gr.Tab("System Status"):
            status_btn = gr.Button("📊 Status aktualisieren")
            status_output = gr.JSON(label="System-Informationen")

            def update_status():
                return get_system_status(components)

            status_btn.click(update_status, outputs=status_output)

        with gr.Tab("Dokumente hinzufügen"):
            with gr.Row():
                file_input = gr.File(
                    label="Dokument hochladen",
                    file_types=[".txt", ".md", ".pdf", ".docx"]
                )
                upload_btn = gr.Button("📄 Hinzufügen")

            upload_output = gr.Textbox(label="Upload-Status")

            def upload_document(file):
                if file is None:
                    return "❌ Keine Datei ausgewählt."

                try:
                    success = add_text_document(components, file.name)
                    if success:
                        return f"✅ Dokument '{file.name}' erfolgreich hinzugefügt."
                    else:
                        return f"⚠️ Dokument '{file.name}' konnte nicht hinzugefügt werden."
                except Exception as e:
                    return f"❌ Fehler beim Upload: {str(e)}"

            upload_btn.click(upload_document, inputs=file_input, outputs=upload_output)

    return interface

# Interface starten
interface = create_gradio_interface(rag_components)
interface.launch(share=True)

# Exkurs: Ähnlichkeitswerte
---

<p><font color='black' size="5">
Wertebereich und Bedeutung
</font></p>

**Ähnlichkeitswerte bewegen sich zwischen 0 und 1:**

- **1.0 = Identisch**: Perfekte Übereinstimmung (sehr selten)
- **0.8 - 0.99 = Sehr ähnlich**: Hohe thematische Übereinstimmung, fast wortgleiche Inhalte
- **0.6 - 0.79 = Ähnlich**: Starke thematische Verbindung, verwandte Konzepte
- **0.4 - 0.59 = Mäßig ähnlich**: Teilweise Übereinstimmung, gemeinsame Themen
- **0.2 - 0.39 = Schwach ähnlich**: Geringe Verbindung, wenige gemeinsame Begriffe
- **0.0 - 0.19 = Nicht ähnlich**: Keine erkennbare thematische Verbindung




<p><font color='black' size="5">
Empfohlene Schwellenwerte
</font></p>

- **Text-Suche**: Mindestens 0.3 für brauchbare Ergebnisse
- **Bild-Suche**: Mindestens 0.2 für relevante Treffer
- **Hochpräzise Suche**: Mindestens 0.6 für sehr spezifische Anfragen



<p><font color='black' size="5">
Praktische Beispiele
</font></p>


**Bei der Frage "Wer ist Thoren Navarro?":**
- Ähnlichkeit 0.76: Dokument enthält direkte Informationen zu Thoren Navarro
- Ähnlichkeit 0.45: Dokument erwähnt Unterwasserarchäologie (verwandtes Thema)
- Ähnlichkeit 0.23: Dokument über Roboter (wenig relevant)



<p><font color='black' size="5">
Berechnungslogik
</font></p>




<p><font color='blue' size="4">
Text-Ähnlichkeit
</font></p>

**Ursprünglicher Score → Ähnlichkeit (similarity):**
```
Ähnlichkeit = 2.0 / (1 + Score)
```

- **Score 0** (perfekte Übereinstimmung) → Ähnlichkeit 1.0
- **Score 0.5** → Ähnlichkeit 0.67
- **Score 1.0** → Ähnlichkeit 0.5
- **Score 2.0** → Ähnlichkeit 0.25

*Der ursprüngliche Score (**Cosine Similarity**, Kosinus-Ähnlichkeit) ist eine Distanz zwischen Embeddings - niedrigere Werte bedeuten ähnlichere Vektoren.*




<p><font color='blue' size="4">
Bild-Ähnlichkeit
</font></p>

**Distanz → Ähnlichkeit:**
```
Ähnlichkeit = max(0, 1 - Distanz)
```

- **Distanz 0** (identisch) → Ähnlichkeit 1.0
- **Distanz 0.3** → Ähnlichkeit 0.7
- **Distanz 0.8** → Ähnlichkeit 0.2
- **Distanz ≥ 1.0** → Ähnlichkeit 0.0

*Die Distanz stammt aus dem CLIP-Modell Vektorvergleich - **Cosinus-Distanz** zwischen Bild-Embeddings. Berechnung: 1 - Cosine Similarity*




<p><font color='black' size="5">
Wichtige Hinweise
</font></p>

- Ähnlichkeitswerte sind **relativ** - sie hängen von der Qualität und Vielfalt der Dokumentensammlung ab
- **Niedrigere Werte** bedeuten nicht unbedingt irrelevante Inhalte, sondern können auf eine diverse Dokumentenbasis hinweisen
- Bei **spezifischen Fachbereichen** können bereits Werte ab 0.2 sehr relevante Informationen enthalten
- **Kontext ist wichtig** - ein Dokument mit 0.4 Ähnlichkeit kann trotzdem die gesuchte Antwort enthalten
- **Verschiedene Modelle** (OpenAI Embeddings vs. CLIP) können unterschiedliche Verteilungen der Ähnlichkeitswerte erzeugen

# A | Aufgaben
---

## A.1 Erweiterte Suchfunktion



Implementieren Sie eine `similarity_search` Funktion, die Dokumente nach Ähnlichkeit zu einem gegebenen Text-Embedding findet:

In [ ]:
def similarity_search(components: RAGComponents,
                     target_embedding: List[float],
                     k: int = 5,
                     threshold: float = 0.8) -> List[Dict[str, Any]]:
    """
    Findet ähnliche Dokumente basierend auf einem Embedding

    Args:
        components: RAG-System-Komponenten
        target_embedding: Ziel-Embedding für Vergleich
        k: Anzahl der zurückzugebenden Ergebnisse
        threshold: Mindest-Ähnlichkeit (0-1)

    Returns:
        Liste von ähnlichen Dokumenten mit Metadaten
    """
    # TODO: Implementierung
    pass

# Test der Funktion
# text_embedding = components.text_embeddings.embed_query("Künstliche Intelligenz")
# similar_docs = similarity_search(rag_components, text_embedding, k=3, threshold=0.7)

## A.2 Batch-Embedding



Erstellen Sie eine Funktion für effiziente Batch-Verarbeitung von Embeddings:

In [ ]:
from typing import Iterator

def batch_create_embeddings(components: RAGComponents,
                           texts: List[str],
                           batch_size: int = 10) -> Iterator[List[List[float]]]:
    """
    Erstellt Embeddings in Batches für bessere Performance

    Args:
        components: RAG-System-Komponenten
        texts: Liste von Texten
        batch_size: Größe der Batches

    Yields:
        Batches von Embeddings
    """
    # TODO: Implementierung mit Batch-Processing
    pass

# Usage Example
# texts = ["Text 1", "Text 2", "Text 3", ...]
# for embedding_batch in batch_create_embeddings(rag_components, texts, batch_size=5):
#     print(f"Verarbeitet {len(embedding_batch)} Embeddings")

## A.3 Evaluations-Framework



Entwickeln Sie ein System zur Bewertung der Suchqualität:

In [ ]:
@dataclass
class EvaluationResult:
    """Ergebnis einer Evaluation"""
    precision: float
    recall: float
    f1_score: float
    avg_response_time: float
    total_queries: int

def evaluate_search_quality(components: RAGComponents,
                          test_queries: List[str],
                          expected_results: List[List[str]]) -> EvaluationResult:
    """
    Evaluiert die Qualität des Suchsystems

    Args:
        components: RAG-System-Komponenten
        test_queries: Liste von Test-Anfragen
        expected_results: Erwartete Ergebnisse pro Anfrage

    Returns:
        Evaluations-Ergebnis mit Metriken
    """
    # TODO: Implementierung der Evaluation
    pass

# Test Data
test_data = [
    ("Wer ist Thoren Navarro?", ["biografien_2.md"]),
    ("Roboter Bilder", ["robot_image.jpg", "cyborg.png"]),
    # ... weitere Test-Fälle
]

# Usage
# queries, expected = zip(*test_data)
# results = evaluate_search_quality(rag_components, queries, expected)
# print(f"F1-Score: {results.f1_score:.3f}")